In [1]:
import hydraseq as hds
from hydraseq import run_convolutions

In [2]:
from collections import namedtuple, defaultdict
Convo = namedtuple('Convo', ['start', 'end', 'pattern', 'lasts', 'nexts'])
endcap = Convo(-1,-1,['end'], [],[])
def to_convo_node(lst_stuff):
    return Convo(lst_stuff[0], lst_stuff[1], lst_stuff[2], [], [])

def link(conv1, conv2):
    conv1.nexts.append(conv2)
    conv2.lasts.append(conv1)

def stackem(lst_convos):
    frame = defaultdict(list)
    ends = []
    for convo in lst_convos:
        if frame[convo[0]]:
            for cnode in frame[convo[0]]:
                convo_node = to_convo_node(convo)
                link(cnode, convo_node)
                ends.append(convo_node)
                if cnode in ends: ends.remove(cnode)
                frame[convo_node.end].append(convo_node)
        else:
            convo_node = to_convo_node(convo)
            ends.append(convo_node)
            frame[convo_node.end].append(convo_node)
    return ends

def recon(end_nodes):
    stack = []
    for node in end_nodes:
        sentence = []
        sentence.append([node.start, node.end, node.pattern])
        while node.lasts:
            node = node.lasts[0]
            sentence.append([node.start, node.end, node.pattern])
        sentence.reverse()
        stack.append(sentence)
    return stack

def pats_only(sentence):
    return [sent[2] for sent in sentence]

In [3]:
sentence = [["spring"], ["leaves"], ["spring"]]
hd0 = hds.Hydraseq("0_")
hd0.insert(sentence)
hd0.columns

defaultdict(list,
            {'leaves': [<node: leaves,(*) spring leaves>],
             'spring': [<node: spring,(*) spring>,
              <node: spring,(*) spring leaves spring>]})

In [4]:
hd1 = hds.Hydraseq("1_")
encodes = [
    "spring 1_Adj",
    "spring 1_Verb",
    "spring 1_Noun",
    "leaves 1_Noun",
    "leaves 1_Verb"
]
for encode in encodes:
    hd1.insert(encode)

In [5]:
hd1.columns

defaultdict(list,
            {'1_Adj': [<node: 1_Adj,(*) spring 1_Adj>],
             '1_Noun': [<node: 1_Noun,(*) spring 1_Noun>,
              <node: 1_Noun,(*) leaves 1_Noun>],
             '1_Verb': [<node: 1_Verb,(*) spring 1_Verb>,
              <node: 1_Verb,(*) leaves 1_Verb>],
             'leaves': [<node: leaves,(*) leaves>],
             'spring': [<node: spring,(*) spring>]})

In [6]:
hd2 = hds.Hydraseq('_')
for pattern in [
    "1_Noun _NP_",
    "1_Adj 1_Noun _NP_",
    "1_Verb _VP_",
    "1_Adj 1_Noun _NP_",
    "1_Adj 1_Adj 1_Adj 1_Noun _NP_",

]:
    hd2.insert(pattern)

In [7]:
conv1 = run_convolutions("spring leaves spring", hd1)
conv1

[[0, 1, ['1_Adj', '1_Noun', '1_Verb']],
 [1, 2, ['1_Noun', '1_Verb']],
 [2, 3, ['1_Adj', '1_Noun', '1_Verb']]]

In [8]:
#lst_convos = [[0,1,['a']], [1,4,['b']], [4,5,['c']],[1,3,['d']],[3,7,['e']], [10,11,['g']]]
sentences = recon(stackem(conv1))
sentences

[[[0, 1, ['1_Adj', '1_Noun', '1_Verb']],
  [1, 2, ['1_Noun', '1_Verb']],
  [2, 3, ['1_Adj', '1_Noun', '1_Verb']]]]

In [9]:
for sent in sentences:
    conv = run_convolutions(pats_only(sent), hd2)
    print(conv)
    for item in recon(stackem(conv)):
        print(item)

[[0, 1, ['_NP_', '_VP_']], [0, 2, ['_NP_']], [1, 2, ['_NP_', '_VP_']], [2, 3, ['_NP_', '_VP_']]]
[[0, 2, ['_NP_']], [2, 3, ['_NP_', '_VP_']]]
[[0, 1, ['_NP_', '_VP_']], [1, 2, ['_NP_', '_VP_']], [2, 3, ['_NP_', '_VP_']]]


In [12]:
ends = stackem([[0, 1, ['a']], [0, 2, ['b']], [1, 2, ['c']], [2, 3, ['d']]])
print(len(ends))
ends[1].lasts[0].pattern

2


['c']